# Analyse Customer Value by Frequency, Recency and Monetary Value

Customer lifetime value (CLV), is the profit margin a company expects to earn over the entirety of their business relationship with the average customer.

Some contributing factors:
- customer churn rate
- retention rate
- sales & marketing strategy

Business may use strategies such as: 
- **Impress** | by quality/pricing
- **Engage**  | by sales & marketing strategies
- **Retain**  | continue to impress and engage

In this notebook, we explore customer segments by *Frequency*, *Recency* and *Monetary Value*

References: 
- [Customer Lifetime Value](https://clevertap.com/blog/customer-lifetime-value/)
- [Frequency, Recency, Monetary Value Analysis](https://clevertap.com/blog/rfm-analysis/) | [whitepaper](https://info.clevertap.com/hubfs/Blog%20Images/A%20Quick%20Start%20Guide%20to%20Automated%20Segmentation%20(1).pdf) 
- [Customer Segmentation Blog](https://towardsdatascience.com/the-most-important-data-science-tool-for-market-and-customer-segmentation-c9709ca0b64a)

## About Data

Source: https://archive.ics.uci.edu/ml/datasets/online+retail#

**Attribute Information**:

`InvoiceNo`: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with **letter 'c'**, it indicates a **cancellation**.

`StockCode`: Product (item) code. Nominal, a 5-digit integral number **uniquely** assigned to each distinct product.

`Description`: Product (item) name. Nominal.

`Quantity`: The quantities of each product (item) per transaction. Numeric.

`InvoiceDate`: Invice Date and time. Numeric, the day and time when each transaction was generated.

`UnitPrice`: Unit price. Numeric, Product price per unit in **sterling**.

`CustomerID`: Customer number. Nominal, a 5-digit integral number **uniquely** assigned to each customer.

`Country`: Country name. Nominal, the name of the country where each **customer resides**.

# Set up

In [ ]:
%load_ext autoreload
%autoreload 2

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Data
## Load Data
Cell below assumed that dataset is registered in AML Workspace.

In [ ]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Workspace, Dataset

# Get workspace information
workspace = Workspace.from_config()
print(workspace.name, workspace.resource_group, workspace.location, workspace.subscription_id, sep='\n')

# Get dataset registered in AML workspace by name
dataset = Dataset.get_by_name(workspace, name='online-retail-processed')

# Convert type Dataset to Pandas Dataframe
df_orig = dataset.to_pandas_dataframe()

## Make a copy

In [ ]:
df = df_orig.copy()

# Get info about 'df'
df.info()

## Prepare Features

### 1. Select Data
- Select data within the range of last 12 months

In [ ]:
# Filter data between two dates
df_sub = df.loc[(df['InvoiceDate'] >= '2011-06-1')
              & (df['InvoiceDate'] < '2011-12-1')].copy()

df_sub.describe(include='all', datetime_is_numeric=True)

Note:
- extreme value
    - `Quantity` : -80995, 80995
    - `UnitPrice` : 38970.00

### 2. Effective Quantity

Assume that we are only interested in effective sales, we will exclude return of goods. For example:

Initially bought,

|InvoiceNo  |StockCode  |Description    |Unit   |Date   |UnitPrice  |TotalAmount    |Country|
|---        |---        |---            |---    |---    |---        |---            |---|
|538370	    |84946	    |ANTIQUE SILVER TEA GLASS ETCHED	|6	|12/12/2010 11:06	|1.25	|16923.0	|United Kingdom|


Then returned,

|InvoiceNo  |StockCode  |Description    |Unit   |Date   |UnitPrice  |TotalAmount    |Country|
|---        |---        |---            |---    |---    |---        |---            |---|
|C538372	|84946	    |ANTIQUE SILVER TEA GLASS ETCHED	|-2	|12/12/2010 11:12	|1.25	|16923.0	|United Kingdom|

Effectively, this customer bought 4 units within that period. 

First, group `df_sub` by `CustomerID`, `StockCode`, `UnitPrice`, `Country`, then, get the sum of `Quantity`, this will give the 'Effective Quantity'.

In [ ]:
# Take the sum when grouped by ['CustomerID', 'StockCode', 'UnitPrice', 'Country']
df_effective_quantity = df_sub.groupby(['CustomerID', 'StockCode', 'UnitPrice', 'Country'], as_index=False, observed=True)['Quantity'].sum() 

# Basic Statstics
df_effective_quantity.describe()

# Display dataframe
df_effective_quantity

#### Check for `df_effective_quantity['Quantity'] < 0`

In [ ]:
# Check for `df_effective_quantity['Quantity'] < 0`
df_effective_quantity[df_effective_quantity['Quantity'] < 0]

#### Remove `CustomerID` where `df_effective_quantity['Quantity']<0`, if exist

In [ ]:
# Define condition
condition = (df_effective_quantity['Quantity']<0)

# Retrieve `CustomerID`, where df_effective_quantity['Quantity']<0
CustomerID_remove = df_effective_quantity[condition]['CustomerID'].unique()

In [ ]:
df_sub.shape

# Remove `CustomerID`, where df_effective_quantity['Quantity']<0
df_sub = df_sub[~df_sub['CustomerID'].isin(CustomerID_remove)]

df_sub.shape

Check that `df_effective_quantity['Quantity'] >= 0`

In [ ]:
# Take the sum when grouped by ['CustomerID', 'StockCode', 'UnitPrice', 'Country']
df_effective_quantity = df_sub.groupby(['CustomerID', 'StockCode', 'UnitPrice', 'Country'], as_index=False, observed=True)['Quantity'].sum() 

# check for effective 'Quantity' < 0
len(df_effective_quantity[df_effective_quantity['Quantity'] < 0])

At this stage, `df_sub`:
- date range 2021-6-1 to 2021-12-1
- contains transactions of `CustomerID` where effective `Quantity` is >= 0

In [ ]:
# Show statistics
df_sub.describe(include='all', datetime_is_numeric=True)

### 3. Derive new features: `TotalSum`
- `TotalSum` : Create a new column to indicate the total sum of an order, i.e. `Quantity` x `UnitPrice`

In [ ]:
# 'TotalSum' = 'Quantity' * 'UnitPrice'
df_sub['TotalSum'] = df_sub['Quantity'] * df_sub['UnitPrice']

# Display 'df_sub'
df_sub

## Analyse Customer Value by *Frequency, Recency, Monetary*

In [ ]:
# Snapshot at latest date of this dataset
snapshot_date = df_sub['InvoiceDate'].max()
snapshot_date

# Calculate Frequency, Recency and Monetary value for each customer
df_frm = df_sub.groupby(['CustomerID']).agg({'InvoiceDate' : lambda x : (snapshot_date - x.max()).days,
                                             'InvoiceNo' : 'count',
                                             'TotalSum' : 'sum'}).rename(columns={'InvoiceDate' : 'Recency(Days)',
                                                                                  'InvoiceNo' : 'Frequency',
                                                                                  'TotalSum' : 'Monetary(£)'})

df_frm

In [ ]:
# Show statistics
df_frm.describe()

### Pair Plot

In [ ]:
_ = sns.pairplot(df_frm, height=2.5, aspect=1.2)
plt.show()

### Map `df_frm` to normal distribution
Pair plot above shows that the distribution for Recency, Frequency, and Monetary are skewed. Here, data transformation is applied to bring the distribution closer to normal distribution. This will aid the subsequent clustering process. 

Data transformation chosen is [PowerTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PowerTransformer.html). You can learn more about mapping one distribution to another via non-linear distribution [here](https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing-transformer).

In [ ]:
from sklearn.preprocessing import PowerTransformer
import pickle

ptransformer = PowerTransformer(method="yeo-johnson")

df_frm_transformed = pd.DataFrame(ptransformer.fit(df_frm).transform(df_frm), 
                                  columns=['Recency(Days)',	'Frequency', 'Monetary(£)'])


if False: 
#if True: # Uncomment to save
    ptransformer_filepath = f'../../.aml/models/powertransformer.pkl'
    pickle.dump(ptransformer, open(ptransformer_filepath, "wb"))

### Register Transformer Model

In [ ]:
#if True: 
if False: # Set 'True' to run code below
    from azure.ai.ml import MLClient
    from azure.ai.ml.entities import Model
    from azure.identity import DefaultAzureCredential

    # get a handle to the workspace
    ml_client = MLClient(credential=DefaultAzureCredential(), 
                        subscription_id=workspace.subscription_id, 
                        resource_group_name=workspace.resource_group, 
                        workspace_name=workspace.name)
    ml_client

    model_filepath = f'../../.aml/models/powertransformer.pkl'

    file_model = Model(
        path = model_filepath,
        name = "powertransformer",
        description = "powertransformer.pkl",
        auto_increment_version = True,)
    
    ml_client.models.create_or_update(file_model)

### Pair Plot

In [ ]:
pplt = sns.pairplot(df_frm_transformed, height=2.5, aspect=1.2)
_ = pplt.fig.suptitle('Normalised Frequency, Recency, Monetary Distribution', y=1.02) # y is position of title
plt.show()

Note that the distribution for Recency, Frequency and Monetary is closer to a normal distribution

# Data Management

## Upload to Blob Storage

In [ ]:
from azureml.core import Workspace, Dataset

datastore = workspace.get_default_datastore()

#if True:
if False: # Set 'True' to run code below
    filename = '../../.aml/data/online-retail-frm.csv'

    # Save to local
    df_frm_transformed.to_csv(filename, index=False)

    # Upload to DataStore
    Dataset.File.upload_directory('../../.aml/data', datastore)

## Register Dataset
Register `df_frm_transformed` as Dataset

In [ ]:
from azureml.core import Workspace, Dataset

datastore = workspace.get_default_datastore()

# if True:
if False: # Set 'True' to run code below

    # Dataset name to register as 
    name = 'online-retail-frm'

    # create a new dataset
    Dataset.Tabular.register_pandas_dataframe(dataframe=df_frm_transformed, 
                                              target=datastore, 
                                              name=name, 
                                              show_progress=True, 
                                              tags={'Purpose':'Tutorial'})